# Coupler Simulation with FDTD tidy3d

tidy3D is a fast GPU based FDTD tool developed by flexcompute.

To run, you need to create an account and add credits. The number of credits that each simulation takes depends on the simulation size and computation time.

In [ ]:
import gdsfactory as gf
import gplugins.tidy3d as gt
import matplotlib.pyplot as plt
import numpy as np
import tidy3d as td
from simulib import materials  # an HHI internal library, not for customers

from hhi import LAYER_STACK

LAYER_STACK.pprint()

nm = 1e-3
wavelength = np.linspace(1300, 1700, 101) * nm
f = td.C_0 / wavelength

## Materials

To use gdsfactory LayerStack for different PDKs into tidy3d you have to create a mapping between each material name from the LayerStack into a tidy3d Medium.

For the HHI E1700 stack, we take InP & SiN from the default tidy3d materials database and for Q(1.06), we use the [simulib materials library](https://gitlab.fe.hhi.de/hogan/simulib/-/blob/master/simulib/materials.py?ref_type=heads), where we have a Broberg/Sellmeier model for $\text{In}_{x}\text{Ga}_{1-x}\text{As}_{y}\text{P}_{1-y}$ alloys.

In [ ]:
# Define a mapping of pdk material names to tidy3d medium objects
mapping = {
    "InP": td.material_library["InP"]["Palik_Lossless"],
    "Q(1.06)": materials.get_tidy3d_medium_isotropic("Q(1.06)", "broberg", wavelength),
    "sin": td.material_library["SiN"]["Horiba"],
}


# Plot the material dispersions

fig, ax = plt.subplots()
ax.set(xlabel="Wavelength (nm)", ylabel=r"$n$")
for material_name, medium in mapping.items():
    eps_complex = medium.eps_model(f)
    n, k = td.Medium.eps_complex_to_nk(eps_complex)
    # Plot the refractive index and extinction coefficient
    ax.plot(wavelength * 1e3, n, label=material_name)

ax.legend()


plt.show()

## Create Component

So get the geometry of the MMI2x2, we can either import a GDS or we can easily create it with gdsfactory.

In [ ]:
CR = gf.components.coupler_ring(cross_section="E1700", gap=1, radius=250)
CR.show()

## Simulation Setup

The gdsfactory plugin for tidy3d allows us to create a simulation object from the component, the material mapping, and HHI LayerStack.

In [ ]:
# Needed since /gplugins/common/base_models/component.py#L205 get_port_layers needs to be fixed
LAYER_STACK.layers["core"] = LAYER_STACK.layers.pop("E1700_3")

# Setup the tidy3d component
c = gt.Tidy3DComponent(
    component=CR,
    layer_stack=LAYER_STACK,
    material_mapping=mapping,  # gt.material_name_to_medium,  # mapping,
    pad_xy_inner=2.0,
    pad_xy_outer=2.0,
    pad_z_inner=0,
    pad_z_outer=0,
    extend_ports=2.0,
)

# plot the component and the layerstack
fig = plt.figure(constrained_layout=True)
gs = fig.add_gridspec(ncols=2, nrows=3, width_ratios=(7, 1))
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[2, 0])
axl = fig.add_subplot(gs[1, 1])
c.plot_slice(x="core", ax=ax0)
c.plot_slice(y="core", ax=ax1)
c.plot_slice(z="core", ax=ax2)
handles, labels = ax0.get_legend_handles_labels()
labels = [
    label.__add__("_" + LAYER_STACK[label].material)
    if label in LAYER_STACK.layers
    else label
    for label in labels
]

axl.legend(handles, labels, loc="center")
axl.axis("off")
plt.show()

# # initialize the tidy3d ComponentModeler
# modeler = c.get_component_modeler(
#     center_z="E1700_3", port_size_mult=(6, 4), sim_size_z=3.0
# )

In [ ]:
c.plot_slice(x="core")

In [ ]:
# initialize the tidy3d ComponentModeler
modeler = c.get_component_modeler(
    center_z="core", port_size_mult=(6, 4), sim_size_z=3.0
)

# we can plot the tidy3d simulation setup
fig, ax = plt.subplots(2, 1)
modeler.plot_sim(z=c.get_layer_center("core")[2], ax=ax[0])
modeler.plot_sim(x=c.ports[0].center[0], ax=ax[1])
fig.tight_layout()
plt.show()